<a href="https://colab.research.google.com/github/KorStats/ocr_receipt/blob/main/%EA%B3%BC%EC%A0%95/(%EB%85%B8%EC%9D%B4)_%EC%9B%90%EB%B3%B8H%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%86%8C%EB%B6%84%EB%A5%98_%ED%95%99%EC%8A%B5%EA%B2%B0%EA%B3%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 1.1 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 2.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595745 sha256=02b68fc91913c392484f6ed907cc693feb9554bb74b4cffbc67f7df8b9015c7d
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 2.1 MB/s 
     |████████████████████████████████| 769 kB 2.0 MB/s 
     |████████████████████████████████| 3.0 MB 68.1 MB/s 
     |████████████████████████████████| 895 kB 78.0 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-8dbbpjwf
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본   산업분류자동화	'Colab Notebooks'  'My Drive'


In [4]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")
ts=pd.read_csv('/mydrive/산업분류자동화/답안 작성용 파일.csv', encoding = "euc-kr")

In [5]:
df=df[df['digit_1']=='H']
ts=ts[ts['digit_1']=='H']

In [6]:
df=df.reset_index(drop=True)

## 데이터 전처리

In [7]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000018,H,52,529,"선박,화물",운송화물,검수
1,id_0000043,H,49,493,개별화물자동차로,고객의 요청으로,화물운송서비스
2,id_0000061,H,49,493,용달화물자동차로,고객의 요청으로,화물운송서비스
3,id_0000063,H,52,529,시청에서,주민의 편의를 위해,주차장운용
4,id_0000103,H,49,493,개별화물자동차로,고객의 요청으로,화물운송서비스
5,id_0000119,H,49,492,개인택시로,일반인을 대상으로,승객운송서비스
6,id_0000120,H,49,492,개인택시로,일반인을 대상으로,승객운송서비스
7,id_0000123,H,49,493,사업장에서,고객의뢰를 받아,물류센터 배송업무
8,id_0000125,H,49,493,개별화물자동차로,고객의 요청으로,화물운송서비스
9,id_0000133,H,49,493,용달화물자동차로,고객의 요청으로,화물운송서비스


In [8]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000018,H,52,529,"선박,화물 운송화물 검수"
1,id_0000043,H,49,493,개별화물자동차로 고객의 요청으로 화물운송서비스
2,id_0000061,H,49,493,용달화물자동차로 고객의 요청으로 화물운송서비스
3,id_0000063,H,52,529,시청에서 주민의 편의를 위해 주차장운용
4,id_0000103,H,49,493,개별화물자동차로 고객의 요청으로 화물운송서비스
...,...,...,...,...,...
98033,id_0999948,H,49,494,사업장에서 도시내에서소화물을 물류배달제공
98034,id_0999961,H,49,493,용달화물자동차로 고객의 요청으로 화물운송서비스
98035,id_0999962,H,49,492,개인택시로 일반인을 대상으로 승객운송서비스
98036,id_0999982,H,49,493,일반화물자동차로 고객의 요청으로 화물운송서비스


In [9]:
# 결측값 확인
df.isnull().sum()

AI_id      0
digit_1    0
digit_2    0
digit_3    0
text       0
dtype: int64

In [10]:
df.shape

(98038, 5)

In [11]:
df['digit_2'].value_counts()

49    90486
52     7196
50      315
51       41
Name: digit_2, dtype: int64

In [12]:
df['digit_3'].value_counts().sort_index()

491       58
492    39176
493    49115
494     2134
495        3
501      281
502       34
511       32
512        9
521     1591
529     5605
Name: digit_3, dtype: int64



#### **추후 과제: 분류가 현재 매우 불균형함 / 정규화를 이용해서 균형을 맞추면 정확도가 올라가는지 확인 필요**


In [13]:
df.loc[(df['digit_3'] == 491), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 492), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 493), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 494), 'digit_3'] = 3  
df.loc[(df['digit_3'] == 495), 'digit_3'] = 4
df.loc[(df['digit_3'] == 501), 'digit_3'] = 5
df.loc[(df['digit_3'] == 502), 'digit_3'] = 6
df.loc[(df['digit_3'] == 511), 'digit_3'] = 7
df.loc[(df['digit_3'] == 512), 'digit_3'] = 8
df.loc[(df['digit_3'] == 521), 'digit_3'] = 9
df.loc[(df['digit_3'] == 529), 'digit_3'] = 10

In [14]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['선박,화물 운송화물 검수', '10'], ['개별화물자동차로 고객의 요청으로 화물운송서비스', '2'], ['용달화물자동차로 고객의 요청으로 화물운송서비스', '2'], ['시청에서 주민의 편의를 위해 주차장운용', '10'], ['개별화물자동차로 고객의 요청으로 화물운송서비스', '2']]


## Train data & test data

In [15]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [17]:
# Setting parameters
max_len = 64
batch_size = 64 #32,16,8로 감소시키기
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [18]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [19]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [20]:
data_train[0]

(array([   2, 2609, 7184,  994, 6999, 7431, 7078,  517, 7601, 6312,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(11, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [21]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [22]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=11,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [23]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [24]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1379 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.2728147506713867 train acc 0.171875
epoch 1 batch id 201 loss 0.27135249972343445 train acc 0.755907960199005
epoch 1 batch id 401 loss 0.0684250146150589 train acc 0.8656873441396509
epoch 1 batch id 601 loss 0.015239009633660316 train acc 0.9053920549084858
epoch 1 batch id 801 loss 0.012248520739376545 train acc 0.9253472222222222
epoch 1 batch id 1001 loss 0.10974051058292389 train acc 0.937859015984016
epoch 1 batch id 1201 loss 0.012729200534522533 train acc 0.94626873438801
epoch 1 train acc 0.9519973281190648


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.991984577922078


  0%|          | 0/1379 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0010691265342757106 train acc 1.0
epoch 5 batch id 201 loss 0.025038428604602814 train acc 0.9964241293532339
epoch 5 batch id 401 loss 0.011849351227283478 train acc 0.9965321072319202
epoch 5 batch id 601 loss 0.001195199671201408 train acc 0.9964642262895175
epoch 5 batch id 801 loss 0.0016540420474484563 train acc 0.9964107365792759
epoch 5 batch id 1001 loss 0.05049809813499451 train acc 0.996363011988012
epoch 5 batch id 1201 loss 0.0013708031037822366 train acc 0.9963311823480433
epoch 5 train acc 0.9963855148658448


  0%|          | 0/154 [00:00<?, ?it/s]

epoch 5 test acc 0.9916801948051948


In [25]:
#학습 모델 저장
torch.save(model, 'H_ori_so_model.pt')